In [1]:
import numpy as np
import pandas as pd

import sys

from os.path import expanduser
sys.path.append(expanduser('~') + '/Lab/Utils/Python/')

from Conversions.translate import *
from Strings.is_a import *
from Math.normalize import z_transform_mode

# Initial setup

In [2]:
paper_pmid = 33082270
paper_name = 'berg_brandl_2020' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

In [5]:
path_to_genes = '../../Private-Utils/datasets_gene2.txt'
path_to_consensus_tested = '../../Private-Utils/yp_2020-09-01_orfs.txt'

# Load & process the data

In [6]:
original_data = pd.read_excel('raw_data/SupplementalFile2_AZCInitialScreenZscores.xlsx', 
                              sheet_name='SupplementalFile1_Zscores', skiprows=5)

In [7]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4845 x 5


In [8]:
original_data['orfs'] = original_data['Systematic Name'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orfs'] = clean_orf(original_data['orfs'])

In [10]:
# Translate to ORFs 
original_data['orfs'] = translate_sc(original_data['orfs'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orfs'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Systematic Name, Average Colony Size on SD, Average Colony Size on SD + AZC, Fitness, Z-Score, orfs]
Index: []


In [12]:
original_data['data'] = original_data['Z-Score']

In [13]:
ix = is_essential(original_data['orfs'])

Loading SGD features from 2017-04-03.


In [14]:
original_data = original_data.loc[~ix.values]

In [15]:
original_data.set_index('orfs', inplace=True)
original_data.index.name='orf'

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(4203, 5)

# Prepare the final dataset

In [18]:
data = original_data[['data']].copy()

In [19]:
dataset_ids = [16665]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,16665
data_type,value
orf,
YAL002W,-0.077941
YAL004W,-0.228766
YAL005C,-0.532464
YAL007C,0.189201
YAL008W,-0.920855


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=original_data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [24]:
data.head()

,dataset_id,16665
,data_type,value
gene_id,orf,
2,YAL002W,-0.077941
1863,YAL004W,-0.228766
4,YAL005C,-0.532464
5,YAL007C,0.189201
6,YAL008W,-0.920855


# Normalize

In [27]:
def normalize_phenotypic_scores(df, has_tested=False):
    
    if not has_tested:
        
        genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
        genes = genes.reset_index().set_index('systematic_name', drop=False)
        
        yp_orfs = pd.read_csv(path_to_consensus_tested, header=None)
        yp_orfs = yp_orfs[1].values
        consensus_tested = [tuple(genes.loc[orf,['id','systematic_name']]) for orf in yp_orfs]
        
        df_index = [tuple(x) for x in df.index]
        
        # Merge consensus tested with current list of ORFs to make sure 
        # we don't miss any gene just because it's not in the current consensus
        consensus_tested = list(set(consensus_tested + df_index))
        
        df = df.reindex(index=consensus_tested, fill_value=0)
        
    df_norm = z_transform_mode(df)
    
    return df_norm

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan

In [31]:
data_all = data.join(data_norm)

In [32]:
data_all.head()

dataset_id          16665          
data_type           value    valuez
gene_id orf                        
2       YAL002W -0.077941 -0.127787
1863    YAL004W -0.228766 -0.281446
4       YAL005C -0.532464 -0.590852
5       YAL007C  0.189201  0.144377
6       YAL008W -0.920855 -0.986544

# Print out

In [33]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [34]:
from IO.save_data_to_db3 import *

In [35]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 33082270...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.00s/it]
